In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer

raw_model_path = "/home/ubuntu/data/exp/proj2410/model/Qwen2.5-0.5B"
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(raw_model_path)
model = AutoModelForCausalLM.from_pretrained(raw_model_path,
    ).to(device)



/home/ubuntu/miniconda3/envs/quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model.model.layers[0].self_attn.q_proj.bias.max()

tensor(47.7500, device='cuda:0', grad_fn=<MaxBackward1>)

In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer

raw_model_path = "/home/ubuntu/data/exp/proj2410/model/Qwen2.5-0.5B"
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(raw_model_path)
model = AutoModelForCausalLM.from_pretrained(raw_model_path,
    ).to(device)

inputs = tokenizer("According to the report, the company has a good", return_tensors="pt").to(device)
outputs = model(**inputs)


/home/ubuntu/miniconda3/envs/quant/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [2]:


def get_max_logits_ids(outputs):
    max_logits, max_ids = outputs.logits.max(dim=-1)
    return max_ids

inputs.input_ids,outputs.logits.shape,get_max_logits_ids(outputs)

(tensor([[11190,   311,   279,  1895,    11,   279,  2813,   702,   264,  1661]],
        device='cuda:0'),
 torch.Size([1, 10, 151936]),
 tensor([[  311,   279,   364,   315,   279,  1372,   594,   220,  2790, 17011]],
        device='cuda:0'))

In [5]:

quant_model_path = "/home/ubuntu/data/exp/proj2410/quant_model/Qwen-2.5-0.5B/EfficientQAT/w2gs128"
from EfficientQAT.quantize.int_linear_real import load_quantized_model,QuantLinear
def get_model_and_tokenizer(_model_path:str) :
    quantized = "w" in _model_path
    if "w" in _model_path:
        wbits =2
        group_size = 128
        model,tokenizer = load_quantized_model(_model_path,
                                                    wbits=wbits,
                                                    group_size=group_size)
    else:
        model = AutoModelForCausalLM.from_pretrained(_model_path)
        tokenizer = AutoTokenizer.from_pretrained(_model_path)
    return model,tokenizer,quantized
quant_model,_,_ = get_model_and_tokenizer(quant_model_path)
quant_model.to("cuda")
quant_output = quant_model(**inputs)

Loading quantized model from /home/ubuntu/data/exp/proj2410/quant_model/Qwen-2.5-0.5B/EfficientQAT/w2gs128


100%|██████████| 24/24 [00:00<00:00, 324.01it/s]


Loading pre-computed quantized weights...
Loading pre-computed quantized weights Successfully


In [8]:
inputs.input_ids[:,1:],get_max_logits_ids(outputs),get_max_logits_ids(quant_output)

(tensor([[ 311,  279, 1895,   11,  279, 2813,  702,  264, 1661]],
        device='cuda:0'),
 tensor([[  311,   279,   364,   315,   279,  1372,   594,   220,  2790, 17011]],
        device='cuda:0'),
 tensor([[  279,   279,  2701,    11,   279,  1372,   702,   264,  2790, 17011]],
        device='cuda:0'))

In [11]:
model.generate(**inputs, max_new_tokens=20),quant_model.generate(**inputs, max_new_tokens=20)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


(tensor([[11190,   311,   279,  1895,    11,   279,  2813,   702,   264,  1661,
          17011,   323,   264,  1661,  2168,    11,   323,   279,  2813,   594,
          17011,   323,  2168,   525,   279, 16266,   369,   279,  2813,   594]],
        device='cuda:0'),
 tensor([[11190,   311,   279,  1895,    11,   279,  2813,   702,   264,  1661,
          17011,   304,   279,  4958,   323,   702,   264,  1661, 17011,   304,
            279,  3081,    13,   576,  2813,   702,   264,  1661, 17011,   304]],
        device='cuda:0'))

In [ ]:
import torch
import torch.nn as nn

linear = nn.Linear(10, 1)
print(f{linear.weight.shape})